In [ ]:

from delta.tables import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.utils import *
from pyspark.sql.window import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
##READ FILES

manual_columns = [
    "cnpj_basico", "cnpj_ordem", "cnpj_dv", "id_matriz_filial", "nome_fantasia", "situacao_cadastral",
    "data_situacao_cadastral",
    "motivo_situacao_cadastral", "nome_cidade_exterior", "pais", "data_inicio_atividade", "cnae_fiscal_princ",
    "cnae_fiscal_sec",
    "tipo_logradouro", "logradouro", "numero", "complemento", "bairro", "cep", "uf", "municipio", "ddd1", "telefone1",
    "ddd2",
    "telefone2", "ddd_fax", "fax", "email", "situacao_especial", "data_situacao_especial"
]

df_estabelecimentos = spark.read.format("csv").option("header", "false").option("sep", ";").load(
    './Estabelecimento/*.csv')
df_estabelecimentos = df_empresas.toDF(*manual_columns)
df_estabelecimentos.createOrReplaceTempView("estabelecimento")

In [ ]:
df_estabelecimento_ajustado = spark.sql('''
    SELECT
        CONCAT(cnpj_basico, cnpj_ordem, cnpj_dv) AS cnpj,
        CASE 
            WHEN id_matriz_filial = 1 THEN 'MATRIZ'
            WHEN id_matriz_filial = 2 THEN 'FILIAL'
        ELSE 
            'DESCONHECIDO'
        END AS tipo_unidade,
        nome_fantasia,
        CASE 
            WHEN situacao_cadastral = '01' THEN 'NULA'
            WHEN situacao_cadastral = '02' THEN 'ATIVA'
            WHEN situacao_cadastral = '03' THEN 'SUSPENSA'
            WHEN situacao_cadastral = '04' THEN 'INAPTA'
            WHEN situacao_cadastral = '08' THEN 'BAIXADA'
        ELSE 
            'DESCONHECIDO'
        END AS situacao_cadastral,
        data_situacao_cadastral,
        motivo_situacao_cadastral,
        nome_cidade_exterior,
        pais,
        data_inicio_atividade,
        cnae_fiscal_princ,
        cnae_fiscal_sec,
        tipo_logradouro,
        logradouro,
        numero,
        complemento,
        bairro,
        cep,
        uf,
        municipio,
        ddd1,
        telefone1,
        ddd2,
        telefone2,
        ddd_fax,
        fax,
        email,
        situacao_especial,
        data_situacao_especial
    FROM
        estabelecimento
''')
df_estabelecimento_ajustado.createOrReplaceTempView("estabelecimento_ajustado")

In [ ]:
df_estabelecimento_ajustado.coalesce(1).write.mode('overwrite').parquet('./Estabelecimento/estabelecimento_parquet')